In [ ]:
pip install cfscrape

In [ ]:
from urllib.request import urlopen #url로부터 웹페이지를 열고 html 를 가져옴
from bs4 import BeautifulSoup #파싱을 도와주는 라이브러리
import requests #HTTP requests 보낼 수 있도록 도와주는 라이브러리
import re #regular expression 도와주는 라이브러리
import pandas as pd
import os

import cfscrape

In [ ]:
#여러 이름으로 변환시켜야 할 gene id가 담긴 file
#띄어쓰기는 #으로, 한줄에 하나의 이름만 있는 txt파일
gene_list = '/content/drive/MyDrive/data/biomedical_informatics/target.txt'
genes = []
temp = open(gene_list, 'r+').read().split('\n')
for i in temp:
  i = i.replace(' ','#')
  genes.append(i)

### 이용하려는 사이트가 아래 2가지 방식으로 링크가 이루어져 최종 링크를 수합하기 위한 코드
- gene 이름으로 바로 세부사이트가 열리는 경우
- gene id를 통해 세부사이트가 열리는 경우 (우리 파일엔 gene id가 없음) => 대부분이 이에 해당함.

In [ ]:
#초기화
index = 1
fin_link = [] # 세부내역까지 볼 수 있는 링크 저장
gene_name = [] # 실제로 검색이 되는 gene만 남긴 것
fail_gene = [] # 접속이 실패한 gene들
gene_id = []

p = re.compile('https://www.ncbi.nlm.nih.gov/gene/\d+') #세부 link를 찾을 regular expression
p2_link = 'https://www.ncbi.nlm.nih.gov'
p2 = re.compile('/gene/\d+') #이 형태로 쓰여진 경우도 있음.

In [ ]:
len(genes)

20075

In [ ]:
for gene in genes[10500:13385]:
  if index%10 == 0:
    print(f'-----{index}/{len(genes[10500:13385])}-----') # 현재 진행상황 프린트
  index += 1
  
  url = f"https://www.ncbi.nlm.nih.gov/gene/?term={gene}" #NCBI 홈페이지에서 gene id에 대해 검색한 결과 페이지를 url에 저장
  print(url)
  scraper = cfscrape.create_scraper()
  response = scraper.get(url)
  #print(response.status_code)
  
  if response.status_code == 200:
      html = response.text
      bsObject = BeautifulSoup(html, 'html.parser')
      flag = False
      result=""
      for link in bsObject.find_all('a'): #읽은 html 코드상 보라색꺼 <meta content="DB00000" ... 인 부분 찾아 저장
        #print(link)
        result = link.get('href') #코드에서 갈색꺼
        try :
          if p.match(result):
            print(result)
            fin_link.append(result)
            gene_name.append(gene)
            gene_id.append(result.split('/')[-1])
            flag = True
            break # 첫번째로 나오는 애가 해당하는 세부 링크임. 하나 찾았으면 나오기
          elif p2.match(result):
            print(p2_link+result)
            fin_link.append(p2_link+result)
            gene_name.append(gene)
            gene_id.append(result.split('/')[-1])
            flag = True
            break
        except TypeError: # 정규표현식으로 검색이 안되는 형식일 때 발생하는 에러 처리
          pass
          #print("<Invalid regular expression %s>" % result)

      if flag != True :
        fin_link.append(url) # 성공했으나 세부 링크가 없었다면, 자체가 세부링크니까 저장
        gene_name.append(gene)
        gene_id.append('')
        
  else : 
      print(f"fail to scrap '{gene}'")
      print(response.status_code)
      fail_gene.append(gene)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
https://www.ncbi.nlm.nih.gov/gene/6836
https://www.ncbi.nlm.nih.gov/gene/?term=SUSD1
https://www.ncbi.nlm.nih.gov/gene/64420
https://www.ncbi.nlm.nih.gov/gene/?term=SUSD5
https://www.ncbi.nlm.nih.gov/gene/26032
https://www.ncbi.nlm.nih.gov/gene/?term=SUV39H1
https://www.ncbi.nlm.nih.gov/gene/6839
https://www.ncbi.nlm.nih.gov/gene/?term=SUV39H2
https://www.ncbi.nlm.nih.gov/gene/79723
https://www.ncbi.nlm.nih.gov/gene/?term=SUV420H1
https://www.ncbi.nlm.nih.gov/gene/51111
https://www.ncbi.nlm.nih.gov/gene/?term=SUV420H2
https://www.ncbi.nlm.nih.gov/gene/84787
https://www.ncbi.nlm.nih.gov/gene/?term=SUZ12
https://www.ncbi.nlm.nih.gov/gene/23512
https://www.ncbi.nlm.nih.gov/gene/?term=SV2A
https://www.ncbi.nlm.nih.gov/gene/9900
https://www.ncbi.nlm.nih.gov/gene/?term=SV2B
https://www.ncbi.nlm.nih.gov/gene/9899
-----480/2885-----
https://www.ncbi.nlm.nih.gov/gene/?term=SV2C
https://www.ncbi.nlm.nih.gov/gene/22987
https://www.ncbi.nlm.nih.gov/gene/?term=SV

In [ ]:
path0 = "/content/drive/MyDrive/data/biomedical_informatics/result" # 결과 저장할 path

# 세부 사이트로 들어가는 주소 통합한 결과 csv file로 저장
T1 = pd.DataFrame({'genename':gene_name[:],'Final link':fin_link[:]})
T1.to_csv(os.path.join(path0,'Final_link_(10500-13385).csv'))

In [ ]:
print("성공한 gene&link pair")
for i in range(len(gene_name)):
  print(gene_name[i],fin_link[i],gene_id[i])

print("링크 찾는것에 실패한 gene")
for i in range(len(fail_gene)):
  print(fail_gene[i])

### 링크 변환 이후 런타임이 끊겼을 경우, 아래 코드 실행
- 링크 변환 csv 파일을 불러오는 과정

In [ ]:
# 런타임 끊겼을 경우, 이전에 저장한 link 불러와서 사용
'''
csv = pd.read_csv('/content/drive/MyDrive/data/biomedical_informatics/result/Final_link_(3500-7000).csv')
gene_name = csv['genename']
fin_link = csv['Final link']
#gene_id = csv['gene_id']
print(gene_name.shape,fin_link.shape)
'''

In [ ]:
# link 저장하는 csv엔 gene_id가 없어서 gene_id 새로 만들어주기.
'''
gene_id = []
index = 1

for i in range(len(fin_link)):
  #NCBI 홈페이지에서 gene id에 대해 검색한 결과 페이지를 url에 저장
  if index%10 == 0:
    print(f'-----{index}/{len(fin_link)}-----') # 현재 진행상황 프린트
  index += 1

  url = fin_link[i]
  print(url,url.split('/')[4])
  if not(url.split('/')[4].startswith('?term=')):
      gene_id.append(url.split('/')[4])
  else :
      gene_id.append('')
  print(gene_id[i])

print(len(gene_id))
'''

### 변환한 링크로 gene name을 ensembl, omim으로 변환한 후, csv 파일로 결과 저장

In [ ]:
ensembl = []
omim = []
fin_gene = []

index = 0

p_ensembl= re.compile('http://www.ensembl.org/id/ENS.*\n*') #Ensembl ID찾을 regular expression
p_omim = re.compile('https://www.ncbi.nlm.nih.gov/omim/\n*') #omim accession number찾을 regular expression

In [ ]:
# 일단 세부링크까지 들어간 이후
'''
<dt>See related</dt>
        <dd>
          <a href="http://www.ensembl.org/id/ENSG00000160783" data-gblink-text="Ensembl" class="genome-browser-link" data-genome-browser-link-sort="5" data-gb-url="http://www.ensembl.org/loc/ENSG00000160783">Ensembl:ENSG00000160783</a>
          <a href="https://www.ncbi.nlm.nih.gov/omim/609176" data-gblink-text="Ensembl" ref="ordinalpos=1&amp;ncbi_uid=11243&amp;link_uid=609176">MIM:609176</a>
'''
for i in range(len(fin_link)):
  #NCBI 홈페이지에서 gene id에 대해 검색한 결과 페이지를 url에 저장
  if i%10 == 0:
    print(f'-----{i}/{len(fin_link)}-----') # 현재 진행상황 프린트

  url = fin_link[i]
  print(url)
  scraper = cfscrape.create_scraper()
  response = scraper.get(url)
  
  #print(response.status_code)
  if response.status_code == 200:
    ensembl.append('')
    omim.append('')
    fin_gene.append(gene_name[i])

    html = response.text
    bsObject = BeautifulSoup(html, 'html.parser')
    flag1 = False
    flag2 = False
    result=""
    for link in bsObject.find_all('a'): #읽은 html 코드상 보라색꺼
      #print(link)      
      result = link.get('href') #코드에서 갈색꺼
      try:
        if p_ensembl.match(result) and flag1==False:
          print(result)
          ensembl[index]=result.split('/')[-1]
          flag1 = True
      except TypeError:
        pass
        #print("<Invalid regular expression %s>" % result)

      try:
        if p_omim.match(result) and flag2==False:
          print(result)
          omim[index]=result.split('/')[-1]
          flag2 = True
      except TypeError:
        pass
        #print("<Invalid regular expression %s>" % result)
      
      if flag1 and flag2: # 두 id를 모두 찾았으면 반복 종료
        break

    index += 1
    # ID 변환을 실패하였을 때,
    if not(flag1 or flag2):
      print(f'{gene_name[i]} 아무것도 못찾았어!!!!!')
      #fail_gene.append(gene_name[i])
    elif flag1 != True :
      print(f'{gene_name[i]} ensembl id 못찾았어!')
    elif flag2 != True :
      print(f'{gene_name[i]} omim id 못찾았어!')   

  else :
    print(f"링크 접속 실패({gene_name[i]})")
    print(response.status_code)
    del gene_id[i]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
https://www.ncbi.nlm.nih.gov/gene/7157
http://www.ensembl.org/id/ENSG00000141510
https://www.ncbi.nlm.nih.gov/omim/191170
https://www.ncbi.nlm.nih.gov/gene/7158
http://www.ensembl.org/id/ENSG00000067369
https://www.ncbi.nlm.nih.gov/omim/605230
https://www.ncbi.nlm.nih.gov/gene/7159
http://www.ensembl.org/id/ENSG00000143514
https://www.ncbi.nlm.nih.gov/omim/602143
https://www.ncbi.nlm.nih.gov/gene/9537
http://www.ensembl.org/id/ENSG00000175274
https://www.ncbi.nlm.nih.gov/omim/617867
https://www.ncbi.nlm.nih.gov/gene/94241
http://www.ensembl.org/id/ENSG00000164938
https://www.ncbi.nlm.nih.gov/omim/606185
https://www.ncbi.nlm.nih.gov/gene/58476
http://www.ensembl.org/id/ENSG00000078804
https://www.ncbi.nlm.nih.gov/omim/617549
https://www.ncbi.nlm.nih.gov/gene/112858
http://www.ensembl.org/id/ENSG00000172315
https://www.ncbi.nlm.nih.gov/omim/608679
https://www.ncbi.nlm.nih.gov/gene/8626
http://www.ensembl.org/id/ENSG00000073282
https://www.ncbi.nlm.nih.

In [ ]:
print(len(fin_gene),len(gene_id),len(ensembl),len(omim))

2885 2885 2885 2885


In [ ]:
path0 = "/content/drive/MyDrive/data/biomedical_informatics/result" # 결과 저장할 path

# id 변환한 결과 csv file로 저장
T2 = pd.DataFrame({'genename':fin_gene[:],'gene_id':gene_id[:], 'Ensembl id':ensembl[:],'omim':omim[:]})
T2.to_csv(os.path.join(path0,'geneID_conversion_(10500-13385).csv'))